# Klasifikacija

Avtokodirniki, so lahko uporabni pri klasifikacijskih problemih. Zakodirani vhodi v latentni prostor se uporabijo kot vektorske vložitve. Preverili bomo kako kvalitetne so vložitve naših naučenih modelov in prstnih odtisov. Pri tem bomo tudi kombinirali različne vložitve.  
Najprej je potrebno naložiti naša modela za avtokodirnik in variacijski avtokodirnik, oba imata enako širino latentnega prostora, ki je 196. Preberemo tudi podatke molekul za prehajanje skozi krvno-možgansko membrano (BBBP.csv), ki jih bomo klasificirali. Pri tem bomo odstranili tiste, ki jih knjižnica RDKit ne more pretvoriti v prstne odtise. V podatkih so molekule, ki so daljše od 120 in vsebujejo večje število znakov, kot sta bila naučena naša modela (več v 010-data.ipnyb). Take molekule bomo še zmerom uporabili. 

In [1]:
import numpy as np
import sys
import itertools
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE, MDS
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold
from xgboost import XGBClassifier
sys.path.insert(0, '../code')
import cheauder_utils
from vae_smiles import CustomVariationalLayer

vae_encoder = cheauder_utils.load_coder_json(
    "../code/model/vae_encoder_196.json",
    "../code/weights/vae_encoder_196_weights.h5",
    custom_objects={'CustomVariationalLayer': CustomVariationalLayer,
    'latent_dim': 196}
)

ae_encoder = cheauder_utils.load_coder_json(
    "../code/model/ae_encoder_196.json",
    "../code/weights/ae_encoder_196_weights.h5",
    custom_objects={'latent_dim': 196}
)

smiles, y, label = cheauder_utils.read_data('../data/BBBP.csv', col_smiles=3, col_target=2, delimiter=',')
fps, y, valid = cheauder_utils.smiles_to_fingerprints(smiles, y)
chars,charset35 = cheauder_utils.load_charset('../code/model/charset_ZINC35.json')
chars,charset36 = cheauder_utils.load_charset('../code/model/charset_ZINC.json')

vae,_,_ = vae_encoder.predict(cheauder_utils.vectorize_smiles(smiles, charset35, 120))
ae = ae_encoder.predict(cheauder_utils.vectorize_smiles(smiles, charset36, 120))
ae, vae = ae[valid], vae[valid]

Using TensorFlow backend.


Po priporočilih na moleculenet.ai smo za te podatke uporabili stratificirano vzorčenje (10 kratno). Uporabili smo klasifikator "extreme boosting gradient" (XGB), metodo podpornih vektorjev (SVM) in naključnih gozdov. "VAE" predstavlja vektorske vložitve našega variacijskega avtokodirnika, "AE" predstavlja vložitve našega avtokodirnika, ostali so pa prstni odtisi z imenom vrste prstnega odtisa. Za kvaliteto klasifikacije smo uporabili metriko AUC ROC.

In [2]:
n_splits = 10
cv = StratifiedKFold(n_splits=n_splits, random_state=0)
svc = SVC(kernel='linear')
xgb = XGBClassifier()
rf = RandomForestClassifier(n_estimators=500)
classifiers = {'SVM': svc, 'XGB': xgb, 'RF': rf}
Xs = {'VAE': vae, 'AE': ae}
for key, value in fps.items():
    Xs[key] = value

Xs['VAE+AE'] = np.hstack((vae,ae))
for keys, values in zip(itertools.product(fps.keys(), ['VAE', 'AE']), itertools.product(fps.values(), [vae,ae])):
    Xs['+'.join(keys)] = np.hstack((values))
Xs['All'] = np.hstack((np.hstack((list(fps.values()))),ae,vae))

cheauder_utils.classification_test(Xs, y, classifiers, n_splits=10);

VAE                | SVM    : 0.721
VAE                | XGB    : 0.705
VAE                | RF     : 0.695
AE                 | SVM    : 0.678
AE                 | XGB    : 0.670
AE                 | RF     : 0.706
topological        | SVM    : 0.743
topological        | XGB    : 0.738
topological        | RF     : 0.728
circular           | SVM    : 0.734
circular           | XGB    : 0.726
circular           | RF     : 0.715
substructure       | SVM    : 0.756
substructure       | XGB    : 0.764
substructure       | RF     : 0.761
avalon             | SVM    : 0.734
avalon             | XGB    : 0.726
avalon             | RF     : 0.714
VAE+AE             | SVM    : 0.756
VAE+AE             | XGB    : 0.728
VAE+AE             | RF     : 0.705
topological+VAE    | SVM    : 0.807
topological+VAE    | XGB    : 0.767
topological+VAE    | RF     : 0.720
topological+AE     | SVM    : 0.770
topological+AE     | XGB    : 0.736
topological+AE     | RF     : 0.711
circular+VAE       | SVM    